In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import random
from collections import Counter

### USE OpenCV KERNEL ###
import cv2
import numpy as np
from scipy.io import loadmat

# Read in .mat Files

In [2]:
# limit number of data points
lim_train = 1000
lim_test = 100

train_data = loadmat('original_data/train_32x32.mat', variable_names='X').get('X')[:,:,:,:lim_train]
train_labels = loadmat('original_data/train_32x32.mat', variable_names='y').get('y')[:lim_train]

test_data = loadmat('original_data/test_32x32.mat', variable_names='X').get('X')[:,:,:,:lim_test]
test_labels = loadmat('original_data/test_32x32.mat', variable_names='y').get('y')[:lim_test]

#extra_data = loadmat('original_data/extra_32x32.mat', variable_names='X').get('X')[:,:,:,:1]
#extra_labels = loadmat('original_data/extra_32x32.mat', variable_names='y').get('y')[:1]

FileNotFoundError: [Errno 2] No such file or directory: 'original_data/train_32x32.mat'

In [ ]:
print('Training data shape: {0} | Training labels shape: {1}'.format(train_data.shape, train_labels.shape))
print('Testing data shape:  {0}  | Testing labels shape:  {1}'.format(test_data.shape, test_labels.shape))
#print('Extra data shape:    {0}     | Extra labels shape:    {1}'.format(extra_data.shape, extra_labels.shape))

In [ ]:
train_data = train_data.transpose((3,0,1,2))
test_data = test_data.transpose((3,0,1,2))
#extra_data = extra_data.transpose((3,0,1,2))

In [ ]:
train_labels = np.ravel(train_labels)
test_labels = np.ravel(test_labels)
#extra_labels = np.ravel(extra_labels)

In [ ]:
print('Training data shape: {0} | Training labels shape: {1}'.format(train_data.shape, train_labels.shape))
print('Testing data shape:  {0}  | Testing labels shape:  {1}'.format(test_data.shape, test_labels.shape))
#print('Extra data shape:    {0}     | Extra labels shape:    {1}'.format(extra_data.shape, extra_labels.shape))

# Looking at One Image

In [ ]:
one_img = train_data[:,:,:,:][0]
one_img.shape

In [ ]:
imgplot = plt.imshow(one_img)

# Conversion with Color Limit

In [ ]:
def gray_conv(image, num_shades):
    '''
    input: image array(h,w), and number of desired gray shades
    output: list of converted rgb values
    '''
    if num_shades < 255 or num_shades > 2:
        x, y, _ = image.shape
        gray = []
        
        for i in range(x):
            for j in range(y):
                conv = 255 / (num_shades - 1)
                avg = (sum(image[i][j])) / 3
                gray.append(int(int(round(avg / conv)) * conv))
    return np.array(gray)

In [ ]:
def img_arr_conv(lst,row,col):
    '''
    ONLY NEEDED FOR VIEWING IMAGES - NOT ANALYSIS
    input: list with desired image height(row) and width(col)
    output: image in the form of ndarray(col,row)
    '''
    result = np.array(lst)
    result = np.ndarray.reshape(result, (row,col))
        
    return result

In [ ]:
def conv_gray_2d_arr(img_arr, col_left, col_right, num_shades):
    '''
    input: array of images as ndarray, left col start and right col end for image slicing
            and number of gray shade limit
    output: ndarray of ndarrays (images, pixels)
    '''
    num_imgs, _, _, _ = img_arr.shape
    
    arr_2d = []
    
    for i in range(num_imgs):
        img = img_arr[:,:,:,:][i]
        img_trim = img[:, col_left:col_right]
        
        gray_img = gray_conv(img_trim, num_shades)
        
        arr_2d.append(gray_img)
    
    return np.asarray(arr_2d)

# Output Numpy Pickles

In [ ]:
train_arr = conv_gray_2d_arr(train_data, col_left=6, col_right=26, num_shades=32)
test_arr = conv_gray_2d_arr(test_data, col_left=6, col_right=26, num_shades=32)

train_arr.dump('generated_data/train_gray_2d.dat')
train_labels.dump('generated_data/train_labels.dat')

test_arr.dump('generated_data/test_gray_2d.dat')
train_labels.dump('generated_data/train_labels.dat')

#conv_gray_2d_arr = numpy.load("conv_gray_2d_arr.dat")

In [ ]:
print(train_arr.shape)
print(train_labels.shape)
print(test_arr.shape)
print(test_labels.shape)

# Demo of Conversion

In [ ]:
img_num = 5
shades = 32

orig_img = train_data[:,:,:,:][img_num]
scaled_img = img_arr_conv(gray_conv(orig_img, shades), 32, 32)
scaled_img = scaled_img[:,6:26]

fig, (ax0, ax1) = plt.subplots(ncols=2, sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})
ax0.imshow(scaled_img, cmap='Greys_r')
ax0.set_title('Mine')
ax0.axis('off')
ax1.imshow(orig_img)
ax1.set_title('Normal')
ax1.axis('off')
plt.show()

# OpenCV Adaptive Threshold

In [ ]:
# retval, thresh = cv2.threshold(a, 4, 255, cv2.THRESH_BINARY)
a = train_data[:,:,:,:][5]
grayscaled = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)

gaus = cv2.adaptiveThreshold(grayscaled, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 1)

# trim to number of focus
gaus = gaus[:,6:26]

plt.imshow(gaus, cmap='Greys_r')
plt.axis('off')
_ = plt.show

In [ ]:
# set training labels of 10 equal to 0
train_labels[train_labels == 10] = 0
test_labels[test_labels == 10] = 0
extra_labels[extra_labels == 10] = 0

In [ ]:
test_data = test_data.transpose((3,0,1,2))
train_labels = np.ravel(train_labels)
test_labels = np.ravel(test_labels)
print(train_data.shape, train_labels.shape)
print(test_data.shape, test_labels.shape)

In [ ]:
image_size = 32  # image height and width
pixel_depth = 255.0  

def im2gray(image):
    '''Normalize images'''
    image = image.astype(float)
    # grayscale conversion per http://www.eyemaginary.com/Rendering/TurnColorsGray.pdf
    image_gray = np.dot(image, [[0.2989],[0.5870],[0.1140]])
    r = np.dot(image, [[0.2989],[0.5870],[0.1140]])
    g = np.dot(image, [[0.5000],[0.5000],[0.1140]])
    b = np.dot(image, [[1.0000],[-1.0000],[0.0000]])
    return image_gray

train_data_c = im2gray(train_data_t)[:,:,:,0]
test_data_c = im2gray(test_data)[:,:,:,0]
valid_data_c = im2gray(valid_data)[:,:,:,0]

print(train_data_c.shape, train_labels_t.shape)
print(test_data_c.shape, test_labels.shape)
print(valid_data_c.shape, valid_labels.shape)

In [ ]:
def GCN(image, min_divisor=1e-4):
    """Global Contrast Normalization"""
    
    imsize = image.shape[0]
    mean = np.mean(image, axis=(1,2), dtype=float)
    std = np.std(image, axis=(1,2), dtype=float, ddof=1)
    std[std < min_divisor] = 1.
    image_GCN = np.zeros(image.shape, dtype=float)
    
    for i in np.arange(imsize):
        image_GCN[i,:,:] = (image[i,:,:] - mean[i]) / std[i]
        
    return image_GCN

train_data_GCN = GCN(train_data_c)
test_data_GCN = GCN(test_data_c)
valid_data_GCN = GCN(valid_data_c)

print(train_data_GCN.shape, train_labels_t.shape)
print(test_data_GCN.shape, test_labels.shape)
print(valid_data_GCN.shape, valid_labels.shape)

In [ ]:
# logistic regression classifier
clf = SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, shuffle=True, verbose=0, 
                    n_jobs=4, random_state=None, learning_rate='optimal')

clf.fit(train_data_c.reshape(train_data_GCN.shape[0],-1), train_labels_t)

train_prediction = clf.predict(train_data_GCN.reshape(train_data_GCN.shape[0],-1))
valid_prediction = clf.predict(valid_data_GCN.reshape(valid_data_GCN.shape[0],-1))

print('Training score is', clf.score(train_data_GCN.reshape(train_data_GCN.shape[0],-1), train_labels_t))
print('Validation score is', clf.score(valid_data_GCN.reshape(valid_data_GCN.shape[0],-1), valid_labels))
print('-'*80)
print('Classification report of training data:\n', classification_report(train_labels_t, train_prediction))
print('Confusion Matrix of training data:\n', confusion_matrix(train_labels_t, train_prediction))

print('Classification report of validation data:\n', classification_report(valid_labels, valid_prediction))
print('Confusion Matrix of validation data:\n', confusion_matrix(valid_labels, valid_prediction))

# SVM

In [ ]:
svm_clf = SVC(gamma=0.001, C=100.)
svm_mdl = svm_clf.fit(train_data_c.reshape(train_data_GCN.shape[0],-1), train_labels_t)

In [ ]:
print('Training score is', svm_mdl.score(train_data_GCN.reshape(train_data_GCN.shape[0],-1), train_labels_t))
print('Validation score is', svm_mdl.score(valid_data_GCN.reshape(valid_data_GCN.shape[0],-1), valid_labels))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(train_labels_t, svm_mdl.predict(train_data_GCN.reshape(train_data_GCN.shape[0],-1)))